# hihello를 말하는 모델 만들기

<img src='./img/1.png'>

In [1]:
import torch
import torch.nn as nn
import torch.functional as F
import torch.optim as optim

### data 준비하기

모델이 자연어를 학습하도록 하려면 자연어를 벡터로 만들어줘야합니다.  
벡터로 만드는 방법은 여러가지가 있으나, 여기서는 가장 간단한 one-hot vector로 만들어 보도록하겠습니다.  

<img src='./img/2.png'>

In [2]:
char_dict = ['h', 'i', 'e' , 'l' , 'o']

x_data = [0, 1, 0, 2, 3, 3] #hihell

x_one_hot = [[[1,0,0,0,0],  #h
              [0,1,0,0,0],  #i
              [1,0,0,0,0],  #h
              [0,0,1,0,0],  #e
              [0,0,0,1,0],  #l
              [0,0,0,1,0]]] #l


y_data = [1, 0, 2, 3, 3, 4] #ihello

In [3]:
# tensor로 변환
inputs = torch.Tensor(x_one_hot)
labels = torch.LongTensor(y_data)

### parameter 정의하기

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

input_size = 5 # vector의 size : one_hot vector의 size는 5
num_classes = 5
hidden_size = 5
batch_size = 1
sequence_length = 6
num_layers = 1 # vanilla RNN이므로 layer는 1개


### Model  준비하기

<img src='./img/3.png' width='400'> <img src='./img/4.png' width = '400'>

In [5]:
class Model(nn.Module):
    def __init__ (self):
        super(Model, self).__init__()
        self.rnn = nn.RNN(input_size = input_size, hidden_size = hidden_size, batch_first = True)
        
    def forward(self, x, hidden):
        x = x.view(batch_size, sequence_length, input_size) # batch_first이므로 batch_size가 먼저 오게끔 reshape
        output, hidden = self.rnn(x, hidden)
        output = output.view(-1, num_classes)
        return output
    
    def init_hidden(self):
        return torch.zeros(num_layers, batch_size, hidden_size).to(device)
        

### Model  만들기

* Loss Function : CrossEntropyLoss
* Optimization : Adam

In [6]:
rnn = Model()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn.parameters(), lr = 0.04)

print("--------Bulid the Model---------")

for epoch in range(100):
    optimizer.zero_grad()
    loss = 0
    hidden = rnn.init_hidden()
    
    outputs = rnn(inputs, hidden)
    
    loss = criterion(outputs, labels)
    
    _, idx = outputs.max(1)
    
    results_str = [char_dict[c] for c in idx.squeeze()]
    
    print("epoch : %d, loss : %1.3f, 예측 결과 : %s" %(epoch+1, loss.data.item(), ''.join(results_str)))
    
    loss.backward()
    optimizer.step()

--------Bulid the Model---------
epoch : 1, loss : 1.787, 예측 결과 : heeeee
epoch : 2, loss : 1.688, 예측 결과 : heeeee
epoch : 3, loss : 1.599, 예측 결과 : heelee
epoch : 4, loss : 1.522, 예측 결과 : eeelee
epoch : 5, loss : 1.455, 예측 결과 : elelel
epoch : 6, loss : 1.397, 예측 결과 : elelll
epoch : 7, loss : 1.348, 예측 결과 : llelll
epoch : 8, loss : 1.305, 예측 결과 : llelll
epoch : 9, loss : 1.265, 예측 결과 : ilelll
epoch : 10, loss : 1.225, 예측 결과 : ilelll
epoch : 11, loss : 1.183, 예측 결과 : ilello
epoch : 12, loss : 1.139, 예측 결과 : ilello
epoch : 13, loss : 1.096, 예측 결과 : ilello
epoch : 14, loss : 1.056, 예측 결과 : ilello
epoch : 15, loss : 1.020, 예측 결과 : ilello
epoch : 16, loss : 0.987, 예측 결과 : ihello
epoch : 17, loss : 0.957, 예측 결과 : ihello
epoch : 18, loss : 0.928, 예측 결과 : ihello
epoch : 19, loss : 0.900, 예측 결과 : ihello
epoch : 20, loss : 0.873, 예측 결과 : ihello
epoch : 21, loss : 0.847, 예측 결과 : ihello
epoch : 22, loss : 0.824, 예측 결과 : ihello
epoch : 23, loss : 0.803, 예측 결과 : ihello
epoch : 24, loss : 0.785, 예측 결과 :